In [1]:
# PyTorch张量计算操作学习笔记
# 涵盖:四则运算、矩阵运算、原地操作、内存管理
import torch

In [2]:
# 1. 四则运算
# PyTorch支持逐元素运算和广播机制
# 广播:当两个张量形状不同时,自动扩展维度进行运算
# 示例:shape(2,3)+shape(3,)会将后者扩展为(1,3)再广播为(2,3)
a = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
b = torch.tensor([[5.0, 6.0], [7.0, 8.0]])

# 张量间加法(逐元素相加)
print(a + b)  # [[1+5, 2+6], [3+7, 4+8]] = [[6,8], [10,12]]

# 张量与标量运算(广播机制)
print(a+10)  # 每个元素+10: [[11,12], [13,14]]

# .add()方法:返回新张量,不修改原张量
print(a.add(10))  # 结果[[11,12], [13,14]], 但a仍是[[1,2], [3,4]]

# .add_()原地操作:带下划线后缀的方法会直接修改原张量,节省内存
# 应用:梯度累积、参数更新等需要原地修改的场景
print(a.add_(10))  # a被修改为[[11,12], [13,14]]
print(a)  # 验证a已被修改

tensor([[ 6.,  8.],
        [10., 12.]])
tensor([[11., 12.],
        [13., 14.]])
tensor([[11., 12.],
        [13., 14.]])
tensor([[11., 12.],
        [13., 14.]])
tensor([[11., 12.],
        [13., 14.]])


In [3]:
# 减法运算
# .sub_()原地减法,恢复a的原始值
# 对应:.sub()不修改原张量, .sub_()修改原张量
print(a.sub_(10))  # a从[[11,12],[13,14]]变回[[1,2],[3,4]]

tensor([[1., 2.],
        [3., 4.]])


In [4]:
# 幂运算
# .pow_(2)计算每个元素的平方
# 应用:计算L2范数、方差等统计量
# 示例:[[1,2],[3,4]]^2 = [[1,4],[9,16]]
print(a.pow_(2))  # 原地平方运算

tensor([[ 1.,  4.],
        [ 9., 16.]])


In [5]:
# 开方运算
# .sqrt_()计算每个元素的平方根
# 这里将[[1,4],[9,16]]开方恢复为[[1,2],[3,4]]
# 应用:标准差计算、向量归一化
print(a.sqrt_())  # 原地开方运算

tensor([[1., 2.],
        [3., 4.]])


In [6]:
# 指数运算(Python运算符)
# 使用**运算符计算e^x, 其中e≈2.7183(自然对数底数)
# 示例:e^[1,2,3] ≈ [2.72, 7.39, 20.09]
# 应用:激活函数(如softmax中的exp运算)
tensor1=torch.tensor([1.0,2,3])
print(2.7183**tensor1)  # [e^1, e^2, e^3]

tensor([ 2.7183,  7.3892, 20.0859])


In [7]:
# 指数运算(PyTorch方法)
# .exp()使用真实的e值计算,结果更精确
# 对比上一cell:这里用math.e,上面用2.7183近似值
# 应用:softmax、log-sum-exp技巧、指数族分布
print(tensor1.exp())  # [e^1, e^2, e^3]精确计算

tensor([ 2.7183,  7.3891, 20.0855])


In [8]:
# 哈达玛积(Hadamard Product):逐元素相乘
# 区别矩阵乘法:对应位置元素相乘,不是线性代数的矩阵乘法
# 示例:[[1,2],[3,4]] ⊙ [[5,6],[7,8]] = [[1*5,2*6],[3*7,4*8]] = [[5,12],[21,32]]
# 应用:注意力机制的mask、dropout、特征融合
tensor1 = torch.tensor([[1,2],[3,4]])
tensor2 = torch.tensor([[5,6],[7,8]])
print(tensor1 * tensor2)  # 运算符方式
print(torch.mul(tensor1,tensor2))  # 函数方式(等价)

tensor([[ 5, 12],
        [21, 32]])
tensor([[ 5, 12],
        [21, 32]])


In [9]:
# 矩阵乘法(Matrix Multiplication):线性代数中的矩阵乘法
# 规则:(m,n)@(n,p)→(m,p), 第一个的列数必须等于第二个的行数
# 计算:结果[i,j] = Σ(A[i,k] * B[k,j])
# 示例:[[1,2],[3,4]] @ [[5,6],[7,8]]
#       →[[1*5+2*7, 1*6+2*8],[3*5+4*7, 3*6+4*8]] = [[19,22],[43,50]]
# 应用:全连接层、注意力分数计算、线性变换
tensor1 = torch.tensor([[1,2],[3,4]])
tensor2 = torch.tensor([[5,6],[7,8]])
print(tensor1 @ tensor2)  # @运算符(推荐)
print(torch.matmul(tensor1,tensor2))  # matmul函数(等价)

# 批量矩阵乘法(Batched Matrix Multiplication)
# 三维张量:(batch, m, n) @ (batch, n, p) → (batch, m, p)
# 每个batch独立进行矩阵乘法
# 应用:Transformer中的批量注意力计算
tensor1 = torch.tensor([[[1,2],[3,4]],[[5,6],[7,8]]])  # shape:(2,2,2)
tensor2 = torch.tensor([[[9,10],[11,12]],[[13,14],[15,16]]])  # shape:(2,2,2)
print(tensor1 @ tensor2)  # batch0:[[1,2],[3,4]]@[[9,10],[11,12]]
print(torch.matmul(tensor1,tensor2))  # batch1:[[5,6],[7,8]]@[[13,14],[15,16]]

tensor([[19, 22],
        [43, 50]])
tensor([[19, 22],
        [43, 50]])
tensor([[[ 31,  34],
         [ 71,  78]],

        [[155, 166],
         [211, 226]]])
tensor([[[ 31,  34],
         [ 71,  78]],

        [[155, 166],
         [211, 226]]])


In [10]:
# 内存分配验证
# Python的id()返回对象的内存地址
# 非原地操作(如X=X+10)会创建新张量,内存地址改变
# 问题:大张量频繁运算会导致内存碎片和性能下降
X = torch.randint(1,10,(3,2,4))
print(id(X))  # 记录原始内存地址
X=X+10  # 创建新张量并重新赋值给X
print(id(X))  # 内存地址已改变(新对象)

2526053193344
2526053197824


In [11]:
# 原地操作节省内存
# 方法1: X = X @ Y (创建新张量,内存地址变)
# 方法2: X += 10 (语法糖,等价于X = X + 10,也创建新张量)
# 方法3: X[:] = X @ Y (切片赋值,原地修改,内存地址不变)
# 应用:训练大模型时减少显存占用
X = torch.randint(1, 10, (3, 2, 4))
Y = torch.randint(1, 10, (3, 4, 1))
print(X)
print(X.shape)  # (3,2,4)
print(id(X))  # 原始地址
print(X @ Y)  # (3,2,4)@(3,4,1)→(3,2,1)

# 注意:X[:]=会自动广播结果(3,2,1)→(3,2,4)以匹配X的形状
# 这会导致数据重复填充(见输出中每行4个元素相同)
#X = X @ Y  # 这会改变地址
# X += 10  # 这也会改变地址
X[:]= X @ Y  # 切片赋值,保持原地址
print(X)
print(X.shape)  # 仍是(3,2,4)
print(id(X))  # 地址未变,证明是原地操作

tensor([[[7, 6, 3, 6],
         [9, 5, 7, 8]],

        [[4, 7, 7, 7],
         [7, 8, 8, 5]],

        [[6, 2, 3, 7],
         [4, 1, 5, 5]]])
torch.Size([3, 2, 4])
2526053197264
tensor([[[129],
         [167]],

        [[137],
         [135]],

        [[111],
         [ 78]]])
tensor([[[129, 129, 129, 129],
         [167, 167, 167, 167]],

        [[137, 137, 137, 137],
         [135, 135, 135, 135]],

        [[111, 111, 111, 111],
         [ 78,  78,  78,  78]]])
torch.Size([3, 2, 4])
2526053197264
